In [1]:
!pwd

/notebooks/argosMHC/LoRA


In [2]:
!python -m venv loravenv

In [3]:
!source loravenv/bin/activate

In [1]:
!pip install --user ipykernel

In [4]:
!python -m ipykernel install --user --name=loraenv

Installed kernelspec loraenv in /root/.local/share/jupyter/kernels/loraenv


In [1]:
# Start
! pip uninstall protobuf -y
!pip install -q transformers==4.24.0 peft==0.10.0 evaluate==0.4.2 protobuf==3.19.6 torch==1.13.0

Found existing installation: protobuf 3.19.6
Uninstalling protobuf-3.19.6:
  Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu116 requires torch==1.12.1, but you have torch 1.13.0 which is incompatible.
torchaudio 0.12.1+cu116 requires torch==1.12.1, but you have torch 1.13.0 which is incompatible.


In [7]:
#sudo apt install git-lfs
!git lfs install

Updated git hooks.
Git LFS initialized.


In [8]:
from utils import compute_metrics

In [9]:
# Load model directly
#from transformers import EsmModel, EsmConfig, EsmPreTrainedModel
from transformers import AutoTokenizer, EsmModel, EsmConfig, EsmPreTrainedModel
import torch
model_name = "../../pre_trained_models/esm2_t36_3B_UR50D/"
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
#model = EsmForMaskedLM.from_pretrained(model_name, local_files_only=True)

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, local_files_only=True, num_labels=2)

Some weights of the model checkpoint at ../../pre_trained_models/esm2_t36_3B_UR50D/ were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'esm.contact_head.regression.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'esm.contact_head.regression.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at ../../pre_trained_models/esm2_t36_3B_UR50D/ and a

In [5]:
import pandas as pd
cols = ["HLA", "peptide", "label", "length", "mhc"]
df = pd.read_csv("../../dataset/hlab/hlab_test.csv",
                header=1,
                names=cols,
                low_memory=False)

In [6]:
df.head()

,HLA,peptide,label,length,mhc
0,HLA-A*01:01,TDKKTHLY,1,8,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
1,HLA-A*01:01,RSDTPLIY,1,8,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
2,HLA-A*01:01,NSDLVQKY,1,8,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
3,HLA-A*01:01,LSDLLDWK,1,8,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
4,HLA-A*01:01,LLQNDGFF,1,8,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY


In [7]:
df.drop(["HLA", "length", "mhc"], axis=1, inplace=True)
df.head()

,peptide,label
0,TDKKTHLY,1
1,RSDTPLIY,1
2,NSDLVQKY,1
3,LSDLLDWK,1
4,LLQNDGFF,1


In [8]:
train_df = df.sample(frac=0.8, random_state=2024)
test_df = df.drop(train_df.index)
train_df.head()

,peptide,label
55731,QSYGNVVEL,1
31984,SRTFSKLDY,1
132987,AHRVPCSCY,0
146586,MGWKGAQAM,0
157254,AARRGADDPP,0


In [9]:
max_length = 512
def encode_text(texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            #return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_text['input_ids'])
        attention_masks.append(encoded_text['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

train_input_ids, train_attention_masks = encode_text(train_df["peptide"])
test_input_ids, test_attention_masks = encode_text(test_df["peptide"])

In [10]:
# Obtener las etiquetas de entrenamiento y prueba
train_labels = torch.tensor(train_df["label"].values)
test_labels = torch.tensor(test_df["label"].values)

In [18]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

config = {
        "lora_alpha": 1, 
        "lora_dropout": 0.5,
        "lr": 3.701568055793089e-04,
        "lr_scheduler_type": "cosine",
        "max_grad_norm": 0.5,
        "num_train_epochs": 1,
        "per_device_train_batch_size": 36,
        "r": 2,
        "weight_decay": 0.3,
        # Add other hyperparameters as needed
    }

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=config["r"],
    lora_alpha=config["lora_alpha"],
    target_modules=[
            "query",
            "key",
            "value",
            "EsmSelfOutput.dense",
            "EsmIntermediate.dense",
            "EsmOutput.dense",
            "EsmContactPredictionHead.regression",
            "classifier"
    ],
    lora_dropout=config["lora_dropout"],
    bias="none",#all, lora_only
)

model = get_peft_model(model, peft_config)

ValueError: Target module EsmClassificationHead(
  (dense): Linear(in_features=2560, out_features=2560, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (out_proj): Linear(in_features=2560, out_features=2, bias=True)
) is not supported. Currently, only the following modules are supported: `torch.nn.Linear`, `torch.nn.Embedding`, `torch.nn.Conv2d`, `transformers.pytorch_utils.Conv1D`.

In [12]:
# Insertamos las matrices A y B en el modelo (get_peft_model)
from peft import get_peft_model
model = get_peft_model(model, lora_config)

ValueError: Please specify `target_modules` in `peft_config`

In [6]:
from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

pretrained_model_generator, input_encoder = load_pretrained_model()
model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(seq_len))
encoded_x = input_encoder.encode_X(seqs, seq_len)
local_representations, global_representations = model.predict(encoded_x, batch_size=batch_size)
# ... use these as features for other tasks, based on local_representations, global_representations

ModuleNotFoundError: No module named 'proteinbert'

In [5]:
seq_length = 50 # for MHC-I
config = ProteinBertConfig.from_pretrained(model_name, num_labels=2 )

model = Trainer(model = BertRnn.from_pretrained(model_name, config=config), compute_metrics = compute_metrics)
test_dataset = DataSetLoaderBERT("../dataset/hlab/hlab_test_micro.csv", max_length=seq_length)


NameError: name 'ProteinBertConfig' is not defined

In [5]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.1 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3


In [12]:
# Load model directly
from transformers import AutoTokenizer, EsmForMaskedLM, pipeline

import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t36_3B_UR50D")

ValueError: Tokenizer class EsmTokenizer does not exist or is not currently imported.

In [13]:
from transformers import AutoTokenizer, EsmModel

import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

ImportError: cannot import name 'EsmModel' from 'transformers' (/usr/local/lib/python3.9/dist-packages/transformers/__init__.py)

In [4]:
! pip freeze > requirements_ult.txt